In [1]:
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import csv
from pandas.tseries.offsets import MonthEnd
from datetime import datetime, timedelta, date
import requests
import json 
import ta

In [4]:
#assign dates to dataframe index
GOOG_df = pd.read_csv("/Users/fahad/project_repo/data/final/GOOG/GOOG_normalised.csv", index_col=0)
dates = GOOG_df.index
GOOG_df['Date'] =dates




In [5]:
GOOG_df['Date'] = pd.to_datetime(GOOG_df.Date, format ="%Y-%m-%d")

In [6]:
GOOG_2010 = pd.read_csv("/Users/fahad/project_repo/data/final/GOOG/GOOG_10yr.csv", index_col=0)

In [7]:
#ADD LABELS TO THE TRAINING AND TEST DATAFRAMES AND RETURN DICTIONARIES WITH DATE:LABEL FORMAT
GOOG_2010_labels = labelling(GOOG_2010)

In [8]:
GOOG_2010_flattened= get_ndays_monthly_flattened(GOOG_2010,GOOG_df,30)

In [9]:
GOOG_2010_labels_df = pd.Series(GOOG_2010_labels).to_frame("Label")

In [10]:
#CREATE DATAFRAME FROM FLATTENED DICTIONARY OF ARRAYS FOR TRAINING AND TEST DATA
GOOG_2010_flat_df = pd.DataFrame.from_dict(GOOG_2010_flattened).T

In [11]:
#JOIN THE LABEL DATAFRAME AND THE FLATTENED DATAFRAME ON THE INDEX WHICH IS THE DATE
GOOG_2010_flat_df = GOOG_2010_flat_df.merge(GOOG_2010_labels_df, left_index=True, right_index=True)

In [12]:
GOOG_2010_flat_df

,0,1,2,3,4,5,6,7,8,9,...,351,352,353,354,355,356,357,358,359,Label
2019-07-31,0.920226,0.913466,0.913208,0.900393,0.019019,0.000000,0.465589,0.344340,0.352485,0.212216,...,0.865887,0.009197,0.402499,0.391972,0.954843,0.970768,0.436838,0.0,0.0,1
2019-06-28,0.960321,0.956079,0.964715,0.949554,0.017513,0.768960,0.567350,0.344340,0.352485,0.482167,...,0.851361,0.016706,0.503188,0.537411,0.963931,0.984841,0.424816,0.0,0.0,0
2019-05-31,0.844734,0.836774,0.843245,0.829070,0.012073,0.224342,0.288770,0.946175,0.979439,0.418772,...,0.912261,0.018459,0.867686,0.814066,0.914507,0.963322,0.499924,0.0,0.0,1
2019-04-30,0.870602,0.865864,0.873471,0.862954,0.011438,0.743645,0.625116,0.935039,0.974785,0.532671,...,0.958603,0.014709,0.624235,0.510807,0.873740,0.942439,0.556151,0.0,0.0,1
2019-03-29,1.000000,1.000000,1.000000,1.000000,0.030188,0.922359,0.733542,0.865761,0.942439,0.663310,...,0.916527,0.015566,0.451951,0.435948,0.978721,0.952925,0.487579,0.0,0.0,0
2019-02-28,0.916331,0.904969,0.912223,0.903786,0.011557,0.266137,0.174752,0.910308,0.909418,0.393740,...,0.865960,0.011386,0.505464,0.463027,0.974955,0.952925,0.436638,0.0,0.0,0
2019-01-31,0.863587,0.861706,0.864345,0.861419,0.011638,0.248630,0.245759,0.981723,0.952925,0.512070,...,0.829992,0.017638,0.475227,0.505580,0.925183,0.982773,0.537434,0.0,0.0,0
2018-12-31,0.832132,0.839923,0.836318,0.839614,0.015411,0.399504,0.528801,0.949447,0.993446,0.530454,...,0.781016,0.028806,0.494104,0.477480,0.887456,0.941159,0.787367,0.0,0.0,0
2018-11-30,0.816771,0.811310,0.808620,0.797619,0.017046,0.437077,0.406871,0.909573,0.954481,0.574845,...,0.802935,0.018877,0.530196,0.453296,0.826121,0.875764,0.779752,0.0,0.0,1
2018-10-31,0.838379,0.842543,0.843828,0.839000,0.017702,0.681094,0.779437,0.870450,0.926529,0.789327,...,0.838733,0.025282,0.608931,0.604376,0.851123,0.865769,0.922310,0.0,0.0,0


In [13]:
export = GOOG_2010_flat_df.to_csv("/Users/fahad/project_repo/data/final/GOOG/GOOG_2010_pre_signals.csv")

In [2]:
def labelling(dataframe):
    """This generates labels for the dataframes we wish to predict price.
    A label of 1 suggests an increase in price between months while a label of 0 indicates
    a decrease in price. If the current price is greater than the previous price then a"""
    empty_dict = dict()
    prev_close =0
    counter = 0
    for index,cols in dataframe.iterrows():
        counter +=1
        current_close = cols['Close']
        if float(prev_close) <= float(current_close):
            #signals an increase in price between last month and next month
            empty_dict[index] = 1
        elif float(prev_close) >= float(current_close):
            empty_dict[index]=0

        #reassign prev close to the current close     
        prev_close = current_close
    
    return empty_dict

In [3]:
def get_ndays_monthly_flattened(dataframe_period_end,dataframe_subperiod,n):
    """returns a dictionary which contains key value pair of the month-end date as key and
    value is the proceeding n days. Returns it as a dictionary of dataframes.
    dataframe period end refers to the dataframe which will act as the period end i.e monthly end date
    sub_period dataframe refers to the dataframe with the preceeding n days, i.e daily"""
    empty_dict = dict()
    count = 0
    for end in dataframe_period_end.index:
#         print ("month end/:%s" % end)
        
        month_key = dataframe_period_end.index[count+1]
        
#         print ("month key:%s" % month_key)
        
        last_index = dataframe_period_end.index[-1]
        
#         print ("last index:%s " % last_index)
        
        if (month_key != last_index):
            temp_df = dataframe_subperiod.loc[dataframe_subperiod['Date']<end].head(n)
#             print ("\n")
#             print (temp_df.head(5))
            temp_df = temp_df.drop(['Date'], axis=1)
            
            if (len(temp_df.values.flatten())==360):
                empty_dict[month_key]= temp_df.values.flatten()
                count +=1      
            else:
                pass

        else:
            pass
       
    return empty_dict
    